In [1]:
import torch
import re
import pandas as pd
import numpy as np
import importlib
import tiktoken
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Building a transformer block


In [2]:


GPT_CONFIG_124M = {
    'vocab_size': 50257,
    'context_length': 1024,
    'emd_dim': 768,
    'n_heads': 12,
    'n_layers': 12,
    'drop_rate': 0.1,
    'qkv_bias': False

}

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, n_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % n_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.n_heads = n_heads
        self.head_dim = d_out // n_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.n_heads, self.head_dim) # Changed from num_heads to n_heads
        values = values.view(b, num_tokens, self.n_heads, self.head_dim) # Changed from num_heads to n_heads
        queries = queries.view(b, num_tokens, self.n_heads, self.head_dim) # Changed from num_heads to n_heads

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

The building blocks: Layer normalization, GELU AF and feed forward nn


In [4]:
class LayerNorm(nn.Module):
  def __init__(self, emd_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emd_dim))
    self.shift = nn.Parameter(torch.zeros(emd_dim))

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim = True)
    var = x.var(dim = -1, keepdim = True, unbiased = False)
    norm_x = (x - mean) / torch.sqrt(var + self.eps)
    return self.scale * norm_x + self.shift

class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3)))) # Converted torch.pi to tensor

class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(cfg['emd_dim'], 4 * cfg['emd_dim']),
        GELU(),
        nn.Linear(4 * cfg['emd_dim'], cfg['emd_dim'])
    )

  def forward(self, x):
    return self.layers(x)

In [5]:
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])

    def forward(self, x):
        for layer in self.layers:
            # Compute the output of the current layer
            layer_output = layer(x)
            # Check if shortcut can be applied
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x


# Transformer Block


In [6]:
class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(
        d_in = cfg['emd_dim'],
        d_out = cfg['emd_dim'],
        # embed_dim = cfg['emd_dim'],
        context_length = cfg['context_length'],
        n_heads = cfg['n_heads'],
        dropout = cfg['drop_rate'], # Changed from drop_rate to dropout
        qkv_bias = cfg['qkv_bias'],
        # batch_first= True, # Removed batch_first as it's not a parameter in MultiHeadAttention
    )
    self.ff = FeedForward(cfg)
    self.ln1 = LayerNorm(cfg['emd_dim'])
    self.ln2 = LayerNorm(cfg['emd_dim'])
    self.drop_shortcut = nn.Dropout(cfg['drop_rate'])

  def forward(self, x):
    shortcut = x
    x = self.ln1(x)
    x= self.att(x) # batch, n_tokens, emd_dim
    x = self.drop_shortcut(x)
    x = x + shortcut # add the original input back

    shortcut = x
    x = self.ln2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut

    return x

In [7]:
# whole GPT-2 model

class GPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emd_dim'])
    self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emd_dim'])
    self.drop_emb = nn.Dropout(cfg['drop_rate'])

    self.trf_blocks = nn.Sequential(
        *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )

    self.final_norm = LayerNorm(cfg['emd_dim'])

    self.out_head = nn.Linear(
        cfg['emd_dim'],
        cfg['vocab_size'],
        bias = False
    )

  def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    tok_embeds = self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))
    x = tok_embeds + pos_embeds # batch, token , emb_dim
    x = self.drop_emb(x)
    x = self.trf_blocks(x) # Corrected attribute name
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits



In [8]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [9]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.1381,  0.0077, -0.1963,  ..., -0.0222, -0.1060,  0.1717],
         [ 0.3865, -0.8408, -0.6564,  ..., -0.5163,  0.2369, -0.3357],
         [ 0.6989, -0.1829, -0.1631,  ...,  0.1472, -0.6504, -0.0056],
         [-0.4290,  0.1669, -0.1258,  ...,  1.1579,  0.5303, -0.5549]],

        [[ 0.1094, -0.2894, -0.1467,  ..., -0.0557,  0.2911, -0.2824],
         [ 0.0882, -0.3552, -0.3527,  ...,  1.2930,  0.0053,  0.1898],
         [ 0.6091,  0.4702, -0.4094,  ...,  0.7688,  0.3787, -0.1974],
         [-0.0612, -0.0737,  0.4751,  ...,  1.2463, -0.3834,  0.0609]]],
       grad_fn=<UnsafeViewBackward0>)


In [10]:
total_params =sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [11]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [12]:
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")

Number of trainable parameters considering weight tying: 124,412,160


In [13]:
total_size_bytes = total_params * 4 #A
total_size_mb = total_size_bytes / (1024 * 1024) #B
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


In [22]:
# predicting the next word in a sequence

def generate_text_word(model, idx, max_new_tokens, context_size):
  # batch, tokens
  for _ in range(max_new_tokens):

    print('idx' , idx.shape)
    idx_cond = idx[:, -context_size:]
    print('idx_cond' , idx_cond.shape)

    with torch.no_grad():
      logits = model(idx_cond)

    print('logits', logits.shape)
    logits = logits[:, -1, :]

    probas = torch.softmax(logits, dim=-1)
    print('probas', probas.shape)

    idx_next = torch.argmax(probas, dim = -1, keepdim = True)

    idx = torch.cat((idx, idx_next), dim = 1)
    print('final idx', idx.shape)

  return idx

In [23]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [24]:
model.eval() #A
out = generate_text_word(
model=model,
idx=encoded_tensor,
max_new_tokens=6,
context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

idx torch.Size([1, 4])
idx_cond torch.Size([1, 4])
logits torch.Size([1, 4, 50257])
probas torch.Size([1, 50257])
final idx torch.Size([1, 5])
idx torch.Size([1, 5])
idx_cond torch.Size([1, 5])
logits torch.Size([1, 5, 50257])
probas torch.Size([1, 50257])
final idx torch.Size([1, 6])
idx torch.Size([1, 6])
idx_cond torch.Size([1, 6])
logits torch.Size([1, 6, 50257])
probas torch.Size([1, 50257])
final idx torch.Size([1, 7])
idx torch.Size([1, 7])
idx_cond torch.Size([1, 7])
logits torch.Size([1, 7, 50257])
probas torch.Size([1, 50257])
final idx torch.Size([1, 8])
idx torch.Size([1, 8])
idx_cond torch.Size([1, 8])
logits torch.Size([1, 8, 50257])
probas torch.Size([1, 50257])
final idx torch.Size([1, 9])
idx torch.Size([1, 9])
idx_cond torch.Size([1, 9])
logits torch.Size([1, 9, 50257])
probas torch.Size([1, 50257])
final idx torch.Size([1, 10])
Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


In [21]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Featureiman Byeswickattribute argue
